<a href="https://colab.research.google.com/github/NikhilDendeti/-Trasformation-Scales-/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install sentence-transformers qdrant-client
!pip install pymupdf



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 45.1 MB/s eta 0:00:00


In [28]:
from google.colab import drive
import fitz
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance
import numpy as np
import re

# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Initialize the path to your PDF document
pdf_path = '/content/drive/My Drive/Insurance Industry _ Report.pdf'

# Step 3: Read the PDF document using PyMuPDF (fitz)
doc = fitz.open(pdf_path)

# Step 4: Function to extract text from PDF and split it into chunks of 500 words
def split_into_chunks_by_words(text, words_per_chunk=500):
    """
    Split the document text into chunks of a specified number of words.
    """
    words = text.split()  # Split text into words
    chunks = []
    current_chunk = []

    for word in words:
        current_chunk.append(word)
        if len(current_chunk) == words_per_chunk:
            chunks.append(" ".join(current_chunk))  # Join the words to form a chunk
            current_chunk = []  # Reset for the next chunk

    # Add any remaining words as the last chunk
    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

# Step 5: Extract text from each page and split it into chunks
doc_content = ""
for page_num in range(len(doc)):
    page = doc.load_page(page_num)
    doc_content += page.get_text()

# Step 6: Split the extracted document content into smaller chunks (500 words per chunk)
chunks = split_into_chunks_by_words(doc_content, words_per_chunk=500)

# Step 7: Initialize Sentence Transformer for document embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

# Step 8: Generate embeddings for each chunk
embeddings = model.encode(chunks)

# Step 9: Set up Qdrant client (using your provided Qdrant instance URL and API key)
qdrant_url = 'https://8db3766a-65c1-409f-aaa9-79d79321e863.europe-west3-0.gcp.cloud.qdrant.io:6333'  # Provided Qdrant URL
api_key = 'xyB4yWC3aXxMfv8VET4cLZdpwMziZ0EM2j3BMQ6gUy--t6G7jVmx8w'  # Provided API Key

# Initialize Qdrant client
client = QdrantClient(url=qdrant_url, api_key=api_key)

# Step 10: Create a collection in Qdrant to store the embeddings if it doesn't exist
collection_name = "google_doc_embeddings"
if not client.collection_exists(collection_name):
    client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=embeddings.shape[1], distance=Distance.COSINE)
    )

# Step 11: Upload the document chunk embeddings to Qdrant
points = [
    {"id": idx + 1, "vector": embedding.tolist(), "payload": {"text": chunk}}
    for idx, (chunk, embedding) in enumerate(zip(chunks, embeddings))
]
client.upsert(collection_name=collection_name, points=points)

# Step 12: Perform a search to find the most suitable chunk for a query
query = "What is insurence company do?"
query_embedding = model.encode([query])[0]

# Perform the search in Qdrant
search_results = client.search(
    collection_name=collection_name,
    query_vector=query_embedding.tolist(),
    limit=3  # Adjust based on how many results you want
)

# Step 13: Display the search results in a structured format
print("\nSearch Results:")
for result in search_results:
    # Show part of the document chunk (first 500 characters) in a more structured format
    document_chunk = result.payload['text']

    # Ensure it is split into lines or paragraphs for better readability
    paragraphs = document_chunk.split("\n")

    # Display the result with improved formatting
    print(f"Score: {result.score:.4f}")
    print("Document Chunk: ")

    # Show the first few paragraphs (or lines) to give a preview
    preview = "\n".join(paragraphs[:5])  # Show first 5 paragraphs (adjust as needed)
    print(preview)

    print("\n" + "-"*50 + "\n")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Search Results:
Score: 0.5281
Document Chunk: 
○​ Insurers must manage diverse regulatory frameworks and 
provide consistent global coverage for multinational 
clients.
Strategic Recommendations
1. Embrace Technology for Operational Efficiency

--------------------------------------------------

Score: 0.5059
Document Chunk: 
○​ Innovation: Telematics-based auto insurance adjusting 
premiums according to driving behavior.
○​ Impact: Rewards safe driving, increasing customer loyalty.
5.​ InsureMO’s PaaS Solutions:​
○​ Innovation: A platform-as-a-service enabling insurers to 

--------------------------------------------------

Score: 0.5011
Document Chunk: 
and Retaliation Employee retaliation after reporting fraud Resolved through settlement Commission Dispute Termination following unpaid commission inquiry Favorable settlement for employee Minority Sharehol